## 講習1 --- IRAFを使ってみる

旧来のIRAFの操作は、ターミナルからCLコマンドラインを使って対話的に行うものです。  
この対話的な操作を、python + Jupyter notebookを使っても行うことができます。
ここでは、IRAFの基本的なタスク、**display, imexam, imstat** を使ってみます。  

### pyrafのための準備  

pythonからIRAFを使うにはpyrafを利用します。  
pyrafを(便利に)使うためには、  

- ホームディレクトリにirafというディレクトリを作成
- そのディレクトリ内で mkirafを実行し、login.cl を作成
- 必要に応じてそのlogin.clを編集  
    (今回のサンプルデータの場合、26行目あたりの'#set stdimage = imt800'を  
    'set stdimage = imt1600'にしておくとよいでしょう。冒頭の#を取り除き、800を1600に)

をしておきます。 こうしておくと、login.clでの設定がpyraf利用時に反映されます。また、~/iraf/uparm/に各タスク(imexam, imstatなど)のパラメータが保存されます。　　

**_補足_** :  ~/.iraf/login.cl がある場合には、そちらの設定が使用されます。

### モジュールの読み込み

In [1]:
from pyraf import iraf 

これでirafのタスクをpythonで関数として使うことができます。(macでは'Warning : sscanf library not ... 'がでることがあります。とりえあず無視してください。)

### サンプルデータ

Unixコマンドを使い、カレントディレクトリおよびその中身を確認することができます。

In [2]:
pwd

'/Users/nakajima/git/tohoku2017python'

In [3]:
ls

README.md
check/
course1_IRAF_1.ipynb
course2_IRAF_2.ipynb
course3_IRAF_3.ipynb
course4_numpy.ipynb
course5_iofits.ipynb
course6_matplotlib.ipynb
exercise.ipynb
sample_data1/
sample_data2/
sample_data3/
上のWikiリンクにも資料あり.md


sample_data1 のディレクトリの中にサンプルの画像FITSファイルがあります。IRAFのコマンド、display, imexam, imstatを使って、それらファイルの表示やデータの吟味、統計量の測定をしてみましょう。

### DS9にFITSデータを表示してみる

別のターミナルからds9を立ち上げておきます。  
irafモジュールのdisplay()関数を使い、引数としてファイルを指定します。

In [4]:
iraf.display('sample_data1/hf0001.fits')

z1=977.7808 z2=1139.195


ds9に、星がいくつか写っている視野が表示されましたね。  

### imexam 
ds9にFITS画像を表示した状態で、次のコマンドを実行し、マウスカーソルを、星のないところにあてて**m**、あるいは星にあてて**a**や**r**をタイプしてみましょう。終わるときには**q**をタイプしてください。  

**m**は、カウスカーソルを当てた部分の5x5ピクセルの範囲のカウント値の統計を表示します。バックグラウンドの値、ばらつきなどを調べるときに使います。  
**a**は、星(点光源)の特徴量(ピーク値、fwhmなど)を表示します。  
**r**は、星のradial profileを表示します。 星がサチっていれば一目でわかります。  

In [5]:
iraf.imexam()

#            SECTION     NPIX     MEAN   MEDIAN   STDDEV      MIN      MAX
   [517:521,573:577]       25    1060.    1058.    15.05    1029.    1097.
#   COL    LINE   COORDINATES
#     R    MAG    FLUX     SKY    PEAK    E   PA BETA ENCLOSED   MOFFAT DIRECT
 588.15  510.25 588.15 510.25
   5.58  13.16  54479.   1058.  10488. 0.12   33 1.62     1.89     1.81   1.86


imexamでは、**r**とか**e**コマンドを使うとさらに別のグラフィックウィンドウが現れます。ブラウザとかの後ろに隠れているかもしれません。

### imstat 
FITS画像の統計量を調べるタスクimstatを使ってみましょう。

In [6]:
iraf.imstat.unlearn()  #  パラメータをデフォルト値に

In [7]:
iraf.imstat('sample_data1/hf0001.fits')

#               IMAGE      NPIX      MEAN    STDDEV       MIN       MAX
 sample_data/hf0001.fits   1048576     893.7     4103.  -199198.    25331.


### eparでパラメータ設定 
**iraf.epar('タスク名')** で、パラメータ設定の画面が別ウィンドウで開きます。GUIでパラメータ設定ができます。
(2017-07 MacOS Sierra + Python3.5.3 + Jupyter Notebook では iraf.epar('display')などでUnicodeDecodeErrorが生じる)

In [8]:
iraf.epar('imstat')

このように、GUIでもパラメータ設定ができるのですが、ここでは次の方法をおすすめします。  


### 変数としてパラメータ設定

Cellでタスクの変数にパラメータを代入してやります。この方法だと、使ったパラメータがこのノートブックに残るので、あとになって「この処理でどんなパラメータ使ったっけ？」となったときに助けになります。

In [9]:
iraf.imstat.fields = 'midpt, mean, stddev'
iraf.imstat.lower = 100
iraf.imstat.upper = 5000

In [10]:
iraf.imstat('sample_data1/hf0001.fits')

#     MIDPT      MEAN    STDDEV
      1058.     1059.     33.93


どんな値がパラメータに入っているかを確認するには、それをprint()してやればよいです。

In [11]:
print (iraf.imstat.fields)

midpt, mean, stddev


### 結果の値を変数へ

上ではimstat()の結果が標準出力に表示されました。それぞれの値を変数に保存するには次のようにします。

In [12]:
out = iraf.imstat('sample_data1/hf0001.fits', format='no', Stdout=1) 
# format='no' でヘッダ行非表示、Stdout=1で戻り値を返す

v = out[0].split()  #  戻り値はリスト
median = float(v[0])   # 文字列をfloatに変換しておく
mean = float(v[1])
stddev = float(v[2])
print (median)
print (mean)
print (stddev)
print (median + 3 * stddev) # floatに変換しておかないとここでおかしなことになる

1058.264
1059.271
33.93
1160.0592199999999


デフォルトではStdout=0なので標準出力に値が返ります。戻り値を変数として返す場合にはStdout=1とします。

### helpドキュメント

このノートブック内でhelpを読むこともできます。  
helpの表示が縦に長すぎる場合、左の余白部分(In[]: の下あたり)をクリックするとスクロールバーつきウインドウ表示になります。
(私の環境でブラウザChromeの場合にはデフォルトでスクロールバー表示されます。)

In [13]:
#iraf.help('imstat')   #  Githubでは表示が長くなってしまうのでコメントアウトしておきます

### 演習1    
sample_data1に別のFITSファイル、hf0002.fitsがあります。  
新しいノートブックファイルを作成し、
1. imexamでバックグラウンドの値とばらつき、星の特徴量、を調べる。
2. imstatでカウント値のメジアンなどを求める。
3. imstatのnclipを2以上にしてみてください。どうなりましたか？それはなぜですか？　　
4. imstatの戻り値の文字列をfloat()で数値に変換しないと、どんなおかしなことになるか、敢えて試してみましょう。 